In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

u'E:\\project\\pychram\\book_scripts'

In [2]:
# input Base CSV数据文件
input_base_index_data_csv = 'E:/project/pychram/traderesp/base/output-csv/output-sh000001-2013-2014-index.csv'
input_base_stock_data_csv = 'E:/project/pychram/traderesp/base/output-csv/output-2013-2014-day-stock-history-fillna-0.csv'

In [3]:
# input strategy CSV数据文件

In [4]:
# 注意：这里请填写数据文件在您电脑中的路径，并注意路径中斜杠的方向
base_index_data = pd.read_csv(input_base_index_data_csv,parse_dates=['date'],encoding='gbk')

In [5]:
base_stock_data = pd.read_csv(input_base_stock_data_csv,parse_dates=['date'],encoding='gbk')

In [6]:
base_index_data.columns

Index([u'date', u'close', u'MA_26', u'close_signal', u'today_signal'], dtype='object')

In [7]:
base_stock_data.columns

Index([u'date', u'code', u'close', u'back_close', u'KDJ_X',
       u'today_kdj_x_signal'],
      dtype='object')

In [8]:
base_stock_data_code_list = base_stock_data['code'].unique()
base_stock_data_code_list = list(base_stock_data_code_list)

#base_stock_data_code_list = [600000,600005]
len(base_stock_data_code_list)

2597

In [9]:
year_list = [(datetime(2013,1,1),datetime(2013,12,31)),(datetime(2014,1,1),datetime(2014,12,31))]

In [10]:
cur_stock_buy_list = {}
stock_buy_sell_history_list = []

In [11]:
def check_stock_buy_list_empty(code):
    global cur_stock_buy_list
    cur_list = cur_stock_buy_list.get(code)
    if cur_list == None:
        # print 'list is none'
        return True
    else:
        if(len(cur_list) <= 0):
            # print 'list count(0 == 0)'
            return True
        else:
            # print 'list count=',len(cur_list)
            return False

In [12]:
def show_cur_msg():
    print "cur_stock_buy_list count() =",len(cur_stock_buy_list)
    print "stock_buy_sell_history_list count() =",len(stock_buy_sell_history_list)

In [13]:
def do_buy_stock(cur_date,cur_code):
    global base_stock_data
    global cur_stock_buy_list

    cur_code = int(cur_code)
    tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) & (base_stock_data['code'] == cur_code)].dropna()
    #print len(tmp_df)
    if len(tmp_df) > 0:
        #print tmp_df['close']
        #type(tmp_df['close'])
        #judge kdj 在金叉上才买
        if int(tmp_df['today_kdj_x_signal']) == 2 :
            buy_price = float(tmp_df['back_close'])
            buy_hand_num = 1
            tmp_buy_rec = {'code':cur_code,
                           'buy_price':buy_price,
                           'buy_num':buy_hand_num*100,
                           'buy_date':cur_date,
                           'buy_money':buy_hand_num*100*buy_price}
            tmp_list = cur_stock_buy_list.get(cur_code)
            if tmp_list == None:
                cur_stock_buy_list[cur_code] = [tmp_buy_rec]
            else:
                cur_stock_buy_list[cur_code].append(tmp_buy_rec)

In [14]:
def do_sell_stock(cur_date,cur_code):
    global base_stock_data
    global cur_stock_buy_list
    global stock_buy_sell_history_list
    
    cur_code = int(cur_code)
    cur_stock_list = cur_stock_buy_list.get(cur_code)
    #print cur_stock_list
    # 当前没有可以卖出的股票
    if (cur_stock_list == None) or (len(cur_stock_list) <= 0):
        return False

    # 遍历列表
    need_delete_list = []
    for item in cur_stock_list:
        code = item['code']
        buy_price =item['buy_price']
        buy_num = item['buy_num']
        buy_money = item['buy_money']
        buy_date = item['buy_date']
        # 查询 当天的股价
        tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) & (base_stock_data['code'] == int(code))].dropna()
        # 当天没有该股票信息,跳过
        if len(tmp_df) == 0:
            #print '没有找到该股票信息',cur_date,code
            continue
        # 
        sell_price = float(tmp_df['back_close'])
        sell_num = buy_num
        sell_money = sell_price * sell_num
        sell_date = cur_date
        #
        need_delete_list.append(item)
        #
        old_item = item
        old_item['sell_price'] = sell_price
        old_item['sell_num'] = sell_num
        old_item['sell_money'] = sell_money
        old_item['sell_date'] = sell_date
        stock_buy_sell_history_list.append(old_item)

    
    #删除
    for item in need_delete_list:
        cur_stock_list.remove(item)
    #设置回 map
    cur_stock_buy_list[cur_code] = cur_stock_list

In [15]:
def do_sell_condition_stock(cur_date,cur_code):
    global base_stock_data
    global cur_stock_buy_list
    global stock_buy_sell_history_list
    
    cur_code = int(cur_code)
    cur_stock_list = cur_stock_buy_list.get(cur_code)
    #print cur_stock_list
    # 当前没有可以卖出的股票
    if (cur_stock_list == None) or (len(cur_stock_list) <= 0):
        return False


    # 遍历列表
    need_delete_list = []
    for item in cur_stock_list:
        code = item['code']
        buy_price =item['buy_price']
        buy_num = item['buy_num']
        buy_money = item['buy_money']
        buy_date = item['buy_date']
        # 查询 当天的股价
        tmp_df = base_stock_data[(base_stock_data['date'] == cur_date) & (base_stock_data['code'] == int(code))].dropna()
        # 当天没有该股票信息,跳过
        if len(tmp_df) == 0:
            #print '没有找到该股票信息',cur_date,code
            continue
            
        sell_price = -1.00
        # 0. 超过 n 天，强制卖出
        if (cur_date - buy_date).days >= 5000:
            sell_price = float(tmp_df['back_close'])
        # 1. n 天内，判断是否死叉    
        else:
            # 1. 查询当天的卖出条件
            # 2. 符合卖出条件，返回当天的股价
            # 2.1 死叉
            if int(tmp_df['today_kdj_x_signal']) == 1:
                sell_price = float(tmp_df['back_close'])
            else:
                continue
            #2.2 盈利过 5%
            #if(float(tmp_df['back_close']) - buy_price)/buy_price > 1.05:
            #    sell_price = float(tmp_df['back_close'])
        #
        
        if sell_price > 0.1 :
            sell_num = buy_num
            sell_money = sell_price * sell_num
            sell_date = cur_date
            #
            need_delete_list.append(item)
            #
            old_item = item
            old_item['sell_price'] = sell_price
            old_item['sell_num'] = sell_num
            old_item['sell_money'] = sell_money
            old_item['sell_date'] = sell_date
            stock_buy_sell_history_list.append(old_item)

    
    #删除
    for item in need_delete_list:
        cur_stock_list.remove(item)
    #设置回 map
    cur_stock_buy_list[cur_code] = cur_stock_list

In [16]:
# 最后一天全部卖出
def end_date_sell(cur_index_data):
    global base_index_data
    global cur_stock_buy_list
    global stock_buy_sell_history_list
    
    end_date = cur_index_data['date'].max()
    for cur_code in cur_stock_buy_list:
        do_sell_stock(end_date,cur_code)

    # 将仍然没有卖出的，记为最后一日卖出,价格等为买入价格
    for cur_code in cur_stock_buy_list:
        tmp_list = cur_stock_buy_list[cur_code]
        for item in tmp_list:
            old_item = item
            old_item['sell_price'] = item['buy_price']
            old_item['sell_num'] = item['buy_num']
            old_item['sell_money'] = item['buy_money']
            old_item['sell_date'] = end_date
            stock_buy_sell_history_list.append(old_item)
        #
        cur_stock_buy_list[cur_code] = []

In [17]:
def write_result_to_csv_file(begtime,endtime,history_list):
    # ========== 将算好的数据输出到csv文件 - 注意：这里请填写输出文件在您电脑中的路径
    # output CSV数据文件
    output = pd.DataFrame(history_list)
    output_stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    output_stock_data_cvs = output_stock_data_path + 'output-stock-history-adjust-m26-kdj-jx-buy-sx-sell-' + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') + '.csv'
    output.to_csv(output_stock_data_cvs,encoding='gbk', index=False)

In [18]:
def run_strategy():
    global base_index_data
    global base_stock_data_code_list
    global year_list
    global cur_stock_buy_list
    global stock_buy_sell_history_list

    for begtime,endtime in year_list:
        cur_base_index_data = base_index_data[(base_index_data[('date')] >= begtime) & (base_index_data['date'] <= endtime)]
        # 遍历时间段
        for id,row in cur_base_index_data.iterrows():
            #空值跳过
            if row.isnull().any():
                continue

            # 当前时间
            cur_date = row['date']
            print cur_date

            # 1.可买状态
            if int(row['today_signal']) == 1 :
                #print row
                #print int(row['today_signal'])
                for cur_code in base_stock_data_code_list:
                    cur_code = int(cur_code)
                    # 判断是否已经买入过
                    if check_stock_buy_list_empty(cur_code) == False:
                        # print cur_date,cur_code,'已经买入过一次，跳过'
                        # 2. check cur_stock_list，看是否有符合卖出条件的 stock
                        do_sell_condition_stock(cur_date,cur_code)
                        continue
                    else:
                        do_buy_stock(cur_date,cur_code)
                        continue

            # 2. 可卖状态        
            if int(row['today_signal']) == 0 :
                #print row
                #print int(row['today_signal'])
                cur_code = int(cur_code)
                for cur_code in base_stock_data_code_list:
                    # 1. 判断当前 stock 列表是否为空
                    # 判断是否已经买入过
                    if check_stock_buy_list_empty(cur_code) == True:
                        # 1.1 当前 stock 列表为空，不需处理,跳过
                        continue
                    # 1.2 当前有 buy_stock list    
                    else:
                        do_sell_stock(cur_date,cur_code)  
                        #
        # 遍历时间段结束
        # 处理最后一天还没有卖出的情况
        end_date_sell(cur_base_index_data)
        write_result_to_csv_file(begtime,endtime,stock_buy_sell_history_list)
        cur_stock_buy_list = {}
        stock_buy_sell_history_list = []
        


In [19]:
run_strategy()

2013-02-18 00:00:00
2013-02-19 00:00:00
2013-02-20 00:00:00
2013-02-21 00:00:00
2013-02-22 00:00:00
2013-02-25 00:00:00
2013-02-26 00:00:00
2013-02-27 00:00:00
2013-02-28 00:00:00
2013-03-01 00:00:00
2013-03-04 00:00:00
2013-03-05 00:00:00
2013-03-06 00:00:00
2013-03-07 00:00:00
2013-03-08 00:00:00
2013-03-11 00:00:00
2013-03-12 00:00:00
2013-03-13 00:00:00
2013-03-14 00:00:00
2013-03-15 00:00:00
2013-03-18 00:00:00
2013-03-19 00:00:00
2013-03-20 00:00:00
2013-03-21 00:00:00
2013-03-22 00:00:00
2013-03-25 00:00:00
2013-03-26 00:00:00
2013-03-27 00:00:00
2013-03-28 00:00:00
2013-03-29 00:00:00
2013-04-01 00:00:00
2013-04-02 00:00:00
2013-04-03 00:00:00
2013-04-08 00:00:00
2013-04-09 00:00:00
2013-04-10 00:00:00
2013-04-11 00:00:00
2013-04-12 00:00:00
2013-04-15 00:00:00
2013-04-16 00:00:00
2013-04-17 00:00:00
2013-04-18 00:00:00
2013-04-19 00:00:00
2013-04-22 00:00:00
2013-04-23 00:00:00
2013-04-24 00:00:00
2013-04-25 00:00:00
2013-04-26 00:00:00
2013-05-02 00:00:00
2013-05-03 00:00:00
